In [ ]:
import numpy as np
import CellModeller
from CellModeller.Simulator import Simulator 

In [ ]:
model = '/Users/timrudge/cellmodeller/Examples/ex1_simpleGrowth.py'
sim = Simulator(model, 0.025, clPlatformNum=0, clDeviceNum=0, saveOutput=True)

In [ ]:
# Run the simulation to ~500 cells
while len(sim.cellStates)<500:
    sim.step()

In [ ]:
print sim.phys.n_cells
print sim.stepNum

# We can get the data directly from the running simulation
cs = sim.cellStates

# or from a saved pickle file
import cPickle
data = cPickle.load(open('/Users/timrudge/cellmodeller/data/ex1_simpleGrowth-17-11-17-14-00/step-00400.pickle','rb'))
cs = data['cellStates']

# Make some convenient data arrays from cell properties
lengths = np.array([cell.length for (id,cell) in cs.iteritems()])
pos = np.array([cell.pos for (id,cell) in cs.iteritems()])
norm = np.array([cell.dir for (id,cell) in cs.iteritems()])

In [ ]:
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np

# Plot the position and orientation of each cell
plt.figure(figsize=(8,8))
plt.plot(pos[:,0], pos[:,1], 'o')
plt.quiver(pos[:,0], pos[:,1], norm[:,0], norm[:,1])

In [ ]:
# Plot histograms of some cell properties

# Length
plt.subplot(121)
plt.title('Length')
plt.hist(lengths)

# Distance from origin
plt.subplot(122)
plt.title('Position')
plt.hist(np.sqrt(pos[:,0]**2+pos[:,1]**2))

In [ ]:
# Compute the cell density in regular bins as a weighted 2d histogram

fig = plt.figure()
H, xedges, yedges = np.histogram2d(pos[:,0], pos[:,1], weights=lengths, bins=10)
plt.imshow(H, interpolation='nearest', origin='low', extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]])
plt.colorbar()

In [ ]:
# We can also examine the details of the running simulation,
# e.g. the residual overlaps between cells (error)

# Get the physics solver
phys = sim.phys
# Update the contact information
phys.find_contacts()
# Overlap per cell
d = 0.5*phys.ct_dists_dev[0:phys.n_cells,:].get()

# Min and max of overlap
print np.min(d.ravel()), np.max(d.ravel())